# Metadata

## CSW

### Definition

"*Catalogue services support the ability to publish and search collections of descriptive information (metadata) for data, services, and related information objects. Metadata in catalogues represent resource characteristics that can be queried and presented for evaluation and further processing by both humans and software. Catalogue services are required to support the discovery and binding to registered information resources within an information community.*"
(*[source: opengeospatial.org](http://www.opengeospatial.org/standards/cat)*)


> Our examples will be based on data and [documentation of the Grand Lyon](https://rdata-grandlyon.readthedocs.io/fr/latest/services.html#service-csw)

### Library

> We use our own Class *CSWGetMetadata* based on the [OWSLib](http://geopython.github.io/OWSLib/) Python library

> We must add that [pycws](https://pycsw.org/) exists. But it seems to heavy for what we want to do. It is partially based on OWSLib (*see [requirements](https://pycsw.org/)*). 

### Example

> We are going to search for "*vélo*" value

#### Parameters & initialise

In [1]:
from get_metadata import CSWGetMetadata as CSW
import pprint

#Parameters
## URL of CSW services
csw_url = "https://download.data.grandlyon.com/catalogue/srv/fre/csw"

## Query
query = "vélo"

## Directory for writing JSON metadata files
directory = "./data/metadata"

## Method
method="csw:AnyText" #default

## Type of query
query_type="equal" #default

## Bbox
bbox=[4.85,45.77,4.9,45.79] #bounding box around La Doua, Lyon, France, default => []

## Element set name (details of response)
esn="full" #default

csw = CSW(
    csw_url,
    query,
    directory,
    method=method,
    query_type=query_type,
    bbox=bbox,
    esn=esn
)

#### Explore some metadata
##### Get all the results keys

In [2]:
csw.get_json()

print ("List of dict keys:\n\n")

for key in csw.dict_.keys():
    print (key)

List of dict keys:


Parcs de stationnement vélos de la Métropole de Lyon
Comptage Vélo de la Métropole de Lyon
Aménagements cyclables  de la Métropole de Lyon
Historique des disponibilités des stations Vélo'v de la Métropole de Lyon
Stations Vélo'v de la Métropole de Lyon (disponibilité temps réel)
Calculateur d'itinéraires multimodaux de la Métropole de Lyon
Pôle d'intérêt - Mode doux
Stations Vélo'v de la Métropole de Lyon
Plan Modes Doux de la Métropole de Lyon


##### Check the JSON of a specific key

With our own Class and functions we can get JSON metadata files by reading CSW services and by adding (*when possible*):
* ***WFS_name***: name to use to request WFS for a specific data
* ***WFS_url***: URL to use to request WFS
* ***geom-type***: by using GeoPandas, we can specify the type of geometry: Point, Polygon, ...

In [3]:
value = list(csw.dict_.keys())[0]

print ("JSON Example for {}:\n\n".format(value))
pprint.pprint (
    csw.dict_[value]
)

JSON Example for Parcs de stationnement vélos de la Métropole de Lyon:


{'WFS_name': 'pvo_patrimoine_voirie.pvostationnementvelo',
 'WFS_url': 'https://download.data.grandlyon.com/wfs/grandlyon',
 'abstract': 'Le parc de stationnement vélos est un objet ponctuel décrivant '
             'le point d implantation des arceaux pour le stationnement des '
             'vélos. Il est caractérisé par des informations de gestion '
             "(adresse, localisation, nb d'arceaux, type de mobilier, etc..)",
 'accessrights': None,
 'alternative': None,
 'bbox': {'BBOX': [5.067, 45.55, 4.681, 45.917], 'EPSG': '3946'},
 'bbox_wgs84': None,
 'contributor': None,
 'coverage': None,
 'created': None,
 'creator': None,
 'date': '2019-04-26T00:25:35',
 'format': None,
 'geom_type': 'Point',
 'identifier': 'a3ae7043-a640-4286-b604-48e4827a2b04',
 'identifiers': [{'identifier': 'a3ae7043-a640-4286-b604-48e4827a2b04',
                  'scheme': None}],
 'ispartof': None,
 'issued': None,
 'language': 

# Data

With our *GetWFS* class (*also based on the OWSLib*), we can harvest the WFS ressource with a specific URL and the right name (*URL and id name are in the metadata dict from our previous operation*)

## Set the Class

In [4]:
from get_data import GetWFS
import geopandas as gpd

wfs_url = csw.dict_[value]["WFS_url"]
version = "2.0.0"
name = csw.dict_[value]["WFS_name"]

wfs = GetWFS(
    wfs_url,
    name=name,
    version=version
)

## Get data and explore first results via a GeoDataframe

In [5]:
wfs.request_url()

# Read data from URL and get GeoDataFrame
gdf = gpd.read_file(wfs.request_GeoJSON)
gdf.head()

,nom,adresse,commune,avancement,gestionnaire,mobiliervelo,localisation,abrite,duree,nbarceaux,capacite,anneerealisation,arceauxprojetes,pole,gid,geometry
0,3014,"52, Rue Baraban entrée patinoire",Lyon 3 ème,Existant,Grand Lyon,Wilmotte,Sur chaussée,Non,Normal,5,10,2015,,,1,POINT (4.868282143077109 45.76178971953962)
1,3009,3 Place Victor Basch entrée métro,Lyon 3 ème,Existant,Grand Lyon,Wilmotte,Sur trottoir,Non,Normal,4,8,2009,,métro,2,POINT (4.848023937370386 45.75367097331701)
2,3031,"194, Rue Garibaldi angle rue de la Part-Dieu",Lyon 3 ème,Existant,Grand Lyon,Wilmotte,Sur chaussée,Non,Normal,4,8,2014,,,3,POINT (4.852076143379453 45.7598676895489)
3,3052,33 Rue Moncey angle avenue de Saxe,Lyon 3 ème,Existant,Grand Lyon,Wilmotte,Sur chaussée,Non,Normal,4,8,2009,,DDE,4,POINT (4.845628601036312 45.75773854171948)
4,3037,99 Rue de la Part-Dieu angle avenue de Saxe,Lyon 3 ème,Existant,Grand Lyon,Wilmotte,Sur chaussée,Non,Normal,5,10,2016,,,5,POINT (4.846081797164236 45.75940793888532)


## Visualise data by using Bokeh capabilities

In [7]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, output_file

from geo_functions import get_figure

output_notebook()

inProj = "epsg:4326"
title = csw.dict_[value]["title"]

p = get_figure(gdf, title, inProj)

show(p)

Loading BokehJS ...